In [147]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

device = 'gpu' if torch.cuda.is_available() else 'cpu'

from genbci.util import init_torch_and_get_device, weights_init, get_exo_subject

epochs_exo = get_exo_data(
    "/Users/daniele/Desktop/thesis/library/genbci/ssvep/data/dataset-ssvep-exoskeleton",
    plot=False,
)


data = epochs_exo.get_data()
labels = epochs_exo.events[:, 2] - 1

T = data.shape[2]


Opening raw data file /Users/daniele/Desktop/thesis/library/genbci/ssvep/data/dataset-ssvep-exoskeleton/./subject12/record-[2014.03.10-19.47.49]_raw.fif...
Isotrak not found
    Range : 0 ... 90527 =      0.000 ...   353.621 secs
Ready.
Reading 0 ... 90527  =      0.000 ...   353.621 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 6 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 6.00, 30.00 Hz: -6.02, -6.02 dB

Opening raw data file /Users/daniele/Desktop/thesis/library/genbci/ssvep/data/dataset-ssvep-exoskeleton/./subject01/record-[2012.07.06-19.02.16]_raw.fif...
Isotrak not found
    Range : 0 ... 57023 =      0.000 ...   222.746 secs
Ready.
Reading 0 ... 57023  =      0.000 ...   222.746 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 6 - 30 Hz

IIR filter par

In [148]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = T
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 8, (1, 8), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(8, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(192, 4)
        

    def forward(self, x):
        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)
        
        # FC Layer
        x = x.view(-1, 192)
        x = self.fc1(x)
        return x


net = EEGNet().to(device)
print(net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 769, 8)).to(device))))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

tensor([[ 0.8146, -3.2390, -1.7555,  1.1818]], grad_fn=<AddmmBackward>)


In [149]:

def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 100
    
    predicted = []
    
    for i in range(len(X)//batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]).to(device))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    inputs = Variable(torch.from_numpy(X).to(device))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':

            predicted = np.argmax(predicted,1)
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results

In [150]:
X_train = np.transpose(data, (0,2,1))
X_train = X_train[:,None,:,:].astype('float32')
y_train = labels.astype('float32')


X_train, X_test, y_train, y_test = train_test_split(
         X_train, y_train, test_size=0.2, random_state=42)


In [151]:
batch_size = 32

for epoch in range(30):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    
    running_loss = 0.0
    for i in range(len(X_train)//batch_size-1):
        s = i*batch_size
        e = i*batch_size+batch_size

        
        inputs = torch.from_numpy(X_train[s:e])
        labels = torch.LongTensor(np.array(y_train[s:e]))
        
        # wrap them in Variable
        inputs, labels = Variable(inputs.to(device)), Variable(labels.to(device))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        
        
        optimizer.step()
        
        running_loss += loss.item()
    
    # Validation accuracy
    params = ["acc"]
    print(params)
    print("Training Loss ", running_loss)
    print("Train - ", evaluate(net, X_train, y_train, params))
    #print("Validation - ", evaluate(net, X_val, y_val, params))
    print("Test - ", evaluate(net, X_test, y_test, params))


Epoch  0
['acc']
Training Loss  32.95331430435181
Train -  [0.2735166425470333]
Test -  [0.2543352601156069]

Epoch  1
['acc']
Training Loss  30.428956747055054
Train -  [0.2981186685962373]
Test -  [0.21965317919075145]

Epoch  2
['acc']
Training Loss  28.851829767227173
Train -  [0.2706222865412446]
Test -  [0.19653179190751446]

Epoch  3
['acc']
Training Loss  29.024805188179016
Train -  [0.2691751085383502]
Test -  [0.1791907514450867]

Epoch  4
['acc']
Training Loss  29.267343878746033
Train -  [0.2821997105643994]
Test -  [0.2947976878612717]

Epoch  5
['acc']
Training Loss  27.91148602962494
Train -  [0.3154848046309696]
Test -  [0.23699421965317918]

Epoch  6
['acc']
Training Loss  27.94833278656006
Train -  [0.32127351664254705]
Test -  [0.23121387283236994]

Epoch  7
['acc']
Training Loss  27.188709378242493
Train -  [0.3111432706222865]
Test -  [0.2774566473988439]

Epoch  8
['acc']
Training Loss  27.480726838111877
Train -  [0.38060781476121563]
Test -  [0.2485549132947976